In [12]:
import geopandas
import urllib
import shapely
import matplotlib.pyplot as plt
from math import sin, cos, sqrt, atan2, radians
import json
from PIL import Image
from dotenv import load_dotenv
import os
import pandas as pd

load_dotenv()

def getDistance(lat1, lon1, lat2, lon2):
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

def getCitySize(cityBounds):
    lat1, lon1, lat2, lon2 = cityBounds
    return getDistance(lat1, lon1, lat2, lon2)

## Using Google Maps and GADM database

In [2]:
world = geopandas.read_file("gadm36.gpkg")
print(world.columns)

Index(['UID', 'GID_0', 'ID_0', 'NAME_0', 'GID_1', 'ID_1', 'NAME_1',
       'VARNAME_1', 'NL_NAME_1', 'HASC_1', 'CC_1', 'TYPE_1', 'ENGTYPE_1',
       'VALIDFR_1', 'VALIDTO_1', 'REMARKS_1', 'GID_2', 'ID_2', 'NAME_2',
       'VARNAME_2', 'NL_NAME_2', 'HASC_2', 'CC_2', 'TYPE_2', 'ENGTYPE_2',
       'VALIDFR_2', 'VALIDTO_2', 'REMARKS_2', 'GID_3', 'ID_3', 'NAME_3',
       'VARNAME_3', 'NL_NAME_3', 'HASC_3', 'CC_3', 'TYPE_3', 'ENGTYPE_3',
       'VALIDFR_3', 'VALIDTO_3', 'REMARKS_3', 'GID_4', 'ID_4', 'NAME_4',
       'VARNAME_4', 'CC_4', 'TYPE_4', 'ENGTYPE_4', 'VALIDFR_4', 'VALIDTO_4',
       'REMARKS_4', 'GID_5', 'ID_5', 'NAME_5', 'CC_5', 'TYPE_5', 'ENGTYPE_5',
       'REGION', 'VARREGION', 'zone', 'geometry'],
      dtype='object')


In [57]:
def googleMap(outline, size="500x400", maptype="terrain", scale=2, zoom=10):
    maps_key = os.getenv("GOOGLE_MAPS_API")
    staticmap_base_url = 'https://maps.googleapis.com/maps/api/staticmap'
    
    # Get the city's name if given such as "Maastricht, NL"
    city = location.split(",")[0]
#     cityData = world[world["NAME_2"]==city]
    
#     print(cityData[["NAME_0", "NAME_1","NAME_2","NAME_3","geometry"]])
#     for x in cityData["geometry"]:
    tolerance = 0.0001
    pathTooLong = True
    while pathTooLong:
        if isinstance(outline, shapely.geometry.Polygon):
            coords_list = list(outline.exterior.coords)
        elif isinstance(outline, shapely.geometry.MultiPolygon):
            coords_list = list(outline.geoms[0].exterior.coords)
        else:
            print("Outline not a shape")

        pathCode = "color:0x0000ff80|weight:1|geodesic:true|fillcolor:0x0000ff80"
        for lat,long in coords_list:
            pathCode = f"{pathCode}|{long},{lat}"
#             print("PathCode length",len(pathCode))

        # Instead of zoom or center give marker of top left and bottom right (x.bounds)
        # Then image fits perfectly
        lat1, long1, lat2, long2 = x.bounds
        distance = getDistance(lat1, long1, lat2, long2)
#             markerCode = f"size:tiny|{x.bounds[3]},{x.bounds[0]}|{x.bounds[1]},{x.bounds[2]}"
        markerCode = f"size:tiny|{long2},{lat1}|{long1},{lat2}"

        # This joins the parts of the URL together into one string.
        url = staticmap_base_url + '?' + urllib.parse.urlencode({
            "center": location,
            "size": size,
            "maptype": maptype,
            "scale": scale,
#                 "zoom": zoom,
            "key": maps_key,
            "markers": markerCode,
            "path": pathCode
        })
        # URLs for all web services are limited to 8192 characters!!
        if len(url) > 8192:
            tolerance += 0.0001
            x = x.simplify(tolerance, preserve_topology=False)
        else:
            pathTooLong = False

#             print("URL length: ",len(url))
    
    image = urllib.request.urlopen(url).read()
    return image, distance


def getGoogleBackgroundMap(cityBounds, size="500x500", maptype="terrain", scale=2):
    maps_key = os.getenv("GOOGLE_MAPS_API")
    staticmap_base_url = 'https://maps.googleapis.com/maps/api/staticmap'
    
    topLeftLat, topLeftLong, botRightLat, botRightLong = cityBounds
    pathCode = f"color:0x0000ff00|{botRightLong},{topLeftLat}|{topLeftLong},{botRightLat}"
    
    url = staticmap_base_url + '?' + urllib.parse.urlencode({
                "size": size,
                "maptype": maptype,
                "scale": scale,
                "key": maps_key,
                "path": pathCode
            })
    image = urllib.request.urlopen(url).read()
    return image

In [47]:
location = "Berlin, Germany"
outline, bounds = getCityOutline(location)
print(outline)

# plt.imshow(citymap)

                                            geometry
0  POLYGON ((13.08835 52.41963, 13.09021 52.41355...
1  POLYGON ((13.50344 52.61899, 13.50365 52.61909...


In [50]:
outline
citymap, distance = googleMap(outline["geometry"])
print("Distance", distance)
with open("citymap.png", "wb") as f:
    f.write(citymap)

Outline not a shape


UnboundLocalError: local variable 'coords_list' referenced before assignment

## Use OpenStreetMap API

In [51]:
def getCityOutline(query: str):
    openStreetMap_base_url = "https://nominatim.openstreetmap.org/search"
    url = openStreetMap_base_url + '?' + urllib.parse.urlencode({
        "q": query,
        "format": "json",
        "polygon_geojson": 1
    })

    response = urllib.request.urlopen(url).read()
    responseJson = json.loads(response)

    # find first multipolygon/polygon result (most likely the correct one)
    for shape in responseJson:
        if isinstance(shapely.geometry.shape(shape["geojson"]), shapely.geometry.MultiPolygon):
            cityShape = shapely.geometry.shape(shape["geojson"])
            lat1, long1, lat2, long2 = cityShape.bounds
            bounds = [lat1, long1, lat2, long2]
#             distance = getDistance(lat1, long1, lat2, long2)
            cityGDF = geopandas.GeoDataFrame({
                "geometry": cityShape
            })
            return cityGDF, bounds
        elif isinstance(shapely.geometry.shape(shape["geojson"]), shapely.geometry.Polygon):
            cityShape = shapely.geometry.shape(shape["geojson"])
            lat1, long1, lat2, long2 = cityShape.bounds
            bounds = [lat1, long1, lat2, long2]
#             distance = getDistance(lat1, long1, lat2, long2)
            cityGDF = geopandas.GeoDataFrame({
                "geometry": cityShape
            }, index=[0])
            return cityGDF, bounds
    # If nothing found return None
    return None, None


def combineMapAndOverlay(googleMapFile, cityOverlayFile):
    googleMap = Image.open(googleMapFile)
    gmWidth, gmHeight = googleMap.size
    cityOverlay = Image.open(cityOverlayFile)
    cityWidth,cityHeight = cityOverlay.size
    # Put overlay in the center
    gmWidth = int(gmWidth / 2)
    gmHeight = int(gmHeight / 2)
    topLeftX = gmWidth - int(cityWidth / 2)
    topLeftY = gmHeight - int(cityHeight / 2)
    botRightX = gmWidth + (cityWidth - int(cityWidth / 2))
    botRightY = gmHeight + (cityHeight - int(cityHeight / 2))
    
    googleMap.paste(cityOverlay, (topLeftX, topLeftY, botRightX, botRightY), mask=cityOverlay)
    googleMap.save("static/images/OverlayWithMaps.png", "png")
    


def overlayCities(city1, city1Bounds, city2, city2Bounds):
    size1 = getCitySize(city1Bounds)
    size2 = getCitySize(city2Bounds)
    if size1 > size2:
        bigCity = city1
        bigSize = size1
        bigCityBounds = city1Bounds
        smallCity = city2
        smallSize = size2
    else:
        bigCity = city2
        bigSize = size2
        bigCityBounds = city2Bounds
        smallCity = city1
        smallSize = size1
    # Save the cities as images
    bigCity.plot(color="red")
    plt.axis("off")
    plt.savefig("static/images/BigCity.png", transparent=True)
    plt.close()
    smallCity.plot(alpha=0.7)
    plt.axis("off")
    plt.savefig("static/images/SmallCity.png", transparent=True)
    plt.close()
    # Get google Maps background of bigger city
    bigCityGooglemap = getGoogleBackgroundMap(bigCityBounds)
    with open("static/images/bigCityGooglemap.png", "wb") as f:
        f.write(bigCityGooglemap)
    
    # Load images with Pillow
    bigCityIm = Image.open("static/images/BigCity.png")
    # Load google Maps background of bigger city and put overlay on top of it
    
    smallCityIm = Image.open("static/images/SmallCity.png")
    oldWidth, oldHeight = smallCityIm.size
    newWidth = int(oldWidth * smallSize / bigSize)
    newHeight = int(oldHeight * smallSize / bigSize)
    # Scale down the smaller city
    smallCityImScaled = smallCityIm.resize((newWidth, newHeight))
    # Find center of bigger city
    bigCityWidth, bigCityHeight = bigCityIm.size
    bigCenterW = int(bigCityWidth / 2)
    bigCenterH = int(bigCityHeight / 2)
    topLeftX = bigCenterW - int(newWidth / 2)
    topLeftY = bigCenterH - int(newHeight / 2)
    botRightX = bigCenterW + (newWidth - int(newWidth / 2))
    botRightY = bigCenterH + (newHeight - int(newHeight / 2))
    # Paste the small city onto the bigger one (the mask key-word reserves the transparency of the png)
    bigCityIm.paste(smallCityImScaled, (topLeftX, topLeftY, botRightX, botRightY), mask=smallCityImScaled)
    bigCityIm.save("static/images/Overlay.png", "png")
    #     bigCityIm.show()
    return bigCityIm

In [58]:
city1Query = "Berlin, Germany"
city2Query = "Maastricht, Netherlands"
city1, bounds1 = getCityOutline(city1Query)
city2, bounds2 = getCityOutline(city2Query)


In [59]:
overlay = overlayCities(city1, bounds1, city2, bounds2)
overlay.show()
print(f"Diagonal size of {city1Query}: {getCitySize(bounds1)}")
print(f"Diagonal size of {city2Query}: {getCitySize(bounds2)}")

Diagonal size of Berlin, Germany: 83.25874486198076
Diagonal size of Maastricht, Netherlands: 18.270753487243397


In [60]:
combineMapAndOverlay("static/images/bigCityGooglemap.png","static/images/BigCity.png")